## 1.14 Example: FIFO Allocation 

In the Finance industry, one needs to fill a sell order from a list of matching buys in a FIFO fashion. Although we state this scenario in terms of buys and sells, it applies equally to general FIFO allocation. We begin with the buys represented as a (time-ordered) list of floats, and a single float sell. 

In [3]:
q)buys:2 1 4 3 5 4f 
q)sell:12f

The objective is to draw successively from the buys until we have exactly filled the sell, then stop. In our case the result we are seeking is,

In [4]:
q)allocation: 2 1 4 3 2 0

The insight is to realize that the cumulative sum of the allocations reaches the sell amount and then levels off: this is an equivalent statement of what it means to do FIFO allocation. 

In [5]:
sums allocation

2 3 7 10 12 12


In [6]:
/We realize that the cumulative sum of buys is the total amount available for allocation at each step. 
sums buys

2 3 7 10 15 19f


In [7]:
/To make this sequence level off at the sell amount, simply use (&)

In [8]:
sell&sums buys 

2 3 7 10 12 12f


Now that we have the cumulative allocation amounts, we need to unwind this to get the step-wise allocations. This entails subtracting successive items in the allocations list.

Wouldn't it be nice if q had a built-in function that returned the successive differences of a numeric list? There is one: deltas and – no surprise – it involves an adverb (called each-previous – more about that in Chapter 5).

In [9]:
deltas 1 2 3 4

1 1 1 1


In [10]:
deltas 10 20 24 23

10 10 4 -1


In [12]:
deltas sell&sums buys

2 1 4 3 2 0f


### Look ma, no loops! 

* Now fasten your seatbelts as we switch on warp drive. In real-world FIFO allocation problems, 
* we actually want to allocate buys FIFO not just to a single sell, but to a sequence of sells. 
* You say, surely this must require a loop.
* Please don't call me Shirley. And no loopy code.

We take buys as before but now we have a list sells, which are to be allocated FIFO from buys.allocations

In [13]:
q)buys:2 1 4 3 5 4f 
q)sells:2 4 3 2 
/The idea is to extend the allocation of buys across multiple sells by considering both the cumulative amounts to be allocated as well as the cumulative amounts available for allocation. 



q)sums[buys]

q)sums[sells]


2 3 7 10 15 19f


2 6 9 11


In [14]:
q)2&sums[buys] 
q)6&sums[buys]
q)9&sums[buys]
q)11&sums[buys]

2 2 2 2 2 2f


2 3 6 6 6 6f


2 3 7 9 9 9f


2 3 7 10 11 11f


Adverbs to the rescue! Our first task requires an adverb that applies a dyadic function and a given right operand to each item of a list on the left. That adverb is called each left and it has the funky notation (\:).

We use it to accomplish in a single operation the four individual (&) operations above. 

In [15]:
q)sums[sells] &\: sums[buys]  

2 2 2 2  2  2 
2 3 6 6  6  6 
2 3 7 9  9  9 
2 3 7 10 11 11


Now we apply (deltas) to unwind the allocation in the vertical direction. 

In [16]:
deltas sums[sells]&\:sums[buys] 

2 2 2 2 2 2
0 1 4 4 4 4
0 0 1 3 3 3
0 0 0 1 2 2


In [17]:
(1 2 3; 10 20)

1 2 3
10 20


In [19]:
deltas (1 2 3; 10 20 10)

1 2  3
9 18 7


In [20]:
count (1 2 3; 10 20 10)

2


In [21]:
count each  (1 2 3; 10 20 10)

3 3


In [22]:
deltas each deltas sums[sells] &\: sums[buys] 

2 0 0 0 0 0
0 1 3 0 0 0
0 0 1 2 0 0
0 0 0 1 1 0



In the context of our allocation problem, we realize that (deltas each) is just the ticket to unwind the remaining cumulative allocation within each row. 

